# Mask R-CNN Demo

A quick intro to using the pre-trained model to detect and segment objects.

In [1]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import re
import cv2
import time
from skimage.measure import find_contours,approximate_polygon,subdivide_polygon
# Root directory of the project
ROOT_DIR = os.path.abspath("../../")
# os.environ['CUDA_VISIBLE_DEVICES']='0,1'
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

MODEL_DIR = os.path.join(ROOT_DIR, "logs")
import skimage.draw as sw

/home/ubuntu/anaconda3/envs/mask/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/mask/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
Using TensorFlow backend.


## Configurations

We'll be using a model trained on the MS-COCO dataset. The configurations of this model are in the ```CocoConfig``` class in ```coco.py```.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the ```CocoConfig``` class and override the attributes you need to change.

In [2]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES= 2
    BATCH_SIZE=1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [123.7 116.8 103.9

## Create Model and Load Trained Weights

In [3]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
# max =0
# dirs = os.listdir(os.path.join(os.getcwd(),'../../logs/coco20180518T1056'))
# freg = re.compile(r'_\d+')
# for dir in dirs:
#     number = freg.search(dir)
#     if number is not None:
#         if int(number.group()[1:])>int(max):
#             max =number.group()[1:]
max = '0040'
COCO_MODEL_PATH='/home/ubuntu/Mask_RCNN/logs/mask_rcnn_coco_'+max+'.h5'
model.load_weights(COCO_MODEL_PATH, by_name=True)

/home/ubuntu/Mask_RCNN/logs/mask_rcnn_coco_0040.h5


## Class Names

The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the MS-COCO dataset, the 'person' class is 1 and 'teddy bear' is 88. The IDs are often sequential, but not always. The COCO dataset, for example, has classes associated with class IDs 70 and 72, but not 71.

To improve consistency, and to support training on data from multiple sources at the same time, our ```Dataset``` class assigns it's own sequential integer IDs to each class. For example, if you load the COCO dataset using our ```Dataset``` class, the 'person' class would get class ID = 1 (just like COCO) and the 'teddy bear' class is 78 (different from COCO). Keep that in mind when mapping class IDs to class names.

To get the list of class names, you'd load the dataset and then use the ```class_names``` property like this.
```
# Load COCO dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)
```

We don't want to require you to download the COCO dataset just to run this demo, so we're including the list of class names below. The index of the class name in the list represent its ID (first class is 0, second is 1, third is 2, ...etc.)

In [4]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
# class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
#                'bus', 'train', 'truck', 'boat', 'traffic light',
#                'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
#                'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
#                'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
#                'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
#                'kite', 'baseball bat', 'baseball glove', 'skateboard',
#                'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
#                'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
#                'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
#                'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
#                'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
#                'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
#                'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
#                'teddy bear', 'hair drier', 'toothbrush']

class_names=['BG','fork', 'knife', 'spoon','hotplate'] # be careful to change this
class_names=['BG','crack']



## Compute ratio

In [5]:
def find_index_of_class(class_ids,class_to_find):
    '''
    find all the index in class_ids given class_to_fidn
    '''
    r = []
    for idx,i in enumerate(class_ids):
        if i == class_to_find:
            r.append(idx)      
    return r

def find_highest_score(scores,class_indexes):
    max_scores = scores[0]
    max_index=class_indexes[0]
    for i in class_indexes:
        if scores[i] > max_scores:
            max_index = i
    index_left= list(set(class_indexes) - set([max_index]))# index of object that isn't highest
    return [max_index],index_left


def find_highest_iou_mask(masks,hotplate_index,other_objects_index):
    '''
    count all the 1 that union from 2 mask
    '''
    hotplate_mask=masks[:,:,hotplate_index]
    highest_index = other_objects_index[0]
    max_union_score = 0
    object_shape = hotplate_mask.shape
    for i in other_objects_index:
        union_score=0
        object_mask = masks[:,:,i]
        for l in range(object_shape[0]):
            for k in range(object_shape[1]):
                if object_mask[l,k]==1 and hotplate_mask[l,k] ==1:
                    union_score +=1
    #print('union score : {} with score {}'.format(union_score,scores[i]))
        if union_score >max_union_score :
            max_union_score=union_score
            highest_index = i
    return [highest_index],union_score


def find_highest_iou_bb(rois,hotplate_index,other_objects_index):
    highest_index = other_objects_index[0]
    max_iou=0
    for i in other_objects_index:
        iou=compute_iou(rois[hotplate_index[0]],rois[i])

        if iou>max_iou:
            max_iou=iou
            highest_index = i
    return [highest_index]



def compute_and_compare_dist(point,centroid,max_dist,max_point):
    point = np.asarray(point)
    centroid = np.asarray(centroid)
    dist=np.linalg.norm(point-centroid)
    if dist>max_dist:
        max_dist=dist
        max_point=point
    return max_dist,max_point

def delete_object(class_to_delete,rois,masks,class_ids,scores):
    for i in sorted(class_to_delete,reverse=True):#sorted reverse to avoid changing index while deleting
        rois = np.delete(rois,i,0)
        masks = np.delete(masks,i,2)
        class_ids = np.delete(class_ids ,i,0)
        scores = np.delete(scores,i,0)
    return rois ,masks ,class_ids ,scores



def find_length_of_object(object_mask,object_roi):
    centroid = [int(object_roi[0]+(object_roi[2]-object_roi[0])/2) , int(object_roi[1]+(object_roi[3]-object_roi[1])/2)]
    max_dist = 0
    max_point =[0,0]
    #use 2 loop to find the polygon and compute distances
    #horizontally search
#     for i in range(object_mask.shape[0]):
#         start = False
#         for j in range(object_mask.shape[1]):
#             if object_mask[i,j]==1:
#                 start=True
#             if object_mask[i,j]==1 and start and object_mask[i,j-1]==0:
#                 max_dist,max_point = compute_and_compare_dist([i,j],centroid,max_dist,max_point)

#             if object_mask[i,j]==1 and start and object_mask[i,j+1]==0:
#                 max_dist,max_point = compute_and_compare_dist([i,j],centroid,max_dist,max_point)
    #vertically search
#     for i in range(0,object_mask.shape[0],3):
#         start = False
#         for j in range(0,object_mask.shape[1],3):
#             if object_mask[i,j]==1:
#                 start =True
#             if object_mask[i,j]==1 and start and object_mask[i-1,j]==0:
#                 max_dist,max_point = compute_and_compare_dist([i,j],centroid,max_dist,max_point)
#             if object_mask[i,j]==1 and start and object_mask[i+1,j]==0:
#                 max_dist,max_point = compute_and_compare_dist([i,j],centroid,max_dist,max_point)
    
    
    padded_mask = np.zeros((object_mask.shape[0] + 2, object_mask.shape[1] + 2), dtype=np.uint8)
    padded_mask[1:-1, 1:-1] =object_mask[:,:,0]
    contours = find_contours(padded_mask,0.5)
    for verts in contours:
#         verts = np.fliplr(verts) - 1
        for point in verts:
            max_dist,max_point = compute_and_compare_dist(point,centroid,max_dist,max_point)
    # need to double the max_dist because max_dist is from center of object 
    return max_dist*2,max_point,centroid


    
def get_tolerance(class_name,contours_len):#compute based on the number of mask
    if class_name == 'hotplate':
        return int(contours_len/80)
    else:
        return 0
    
def get_ratio_and_refine_hotplate(r,class_names,threshold):
    '''
    compare the ratio between hotplate and object
    in case detect more than 1 hotplate -> choose the highest score
    '''
    #delete index under threshold
    index_to_delete =[]
    for idx,score in enumerate(r['scores']):
        if score < threshold:
            index_to_delete.append(idx)
    r['rois'],r['masks'],r['class_ids'],r['scores'] = delete_object(index_to_delete,r['rois'],r['masks'],r['class_ids'],r['scores'])


    
    if len(r['class_ids']) <=1:
        print('not enough object to compute ratio')
        return r,None,None,None
    else:
        hotplate_index = find_index_of_class(r['class_ids'], class_names.index('hotplate'))

        # if there is no hotplate 
        if len(hotplate_index) == 0:
            return r,None,None,None
        
        # if detect more than 1 hotplate
        if len(hotplate_index)>1:
            hotplate_index,other_index = find_highest_score(r['scores'],hotplate_index)
            r['rois'],r['masks'],r['class_ids'],r['scores']=  delete_object(other_index,r['rois'],r['masks'].astype(int),r['class_ids'],r['scores'])
            
        # find object to compute ratio
        reference_object_index = list(set(range(len(r['class_ids'])))-set(hotplate_index))
        
        if len(reference_object_index) == 0 :
            print('not enough object to compute ratio')
            return r,None,None,None
        
        # select reference object index manually
#         if len(reference_object_index)>1:
#             object_index,union_score= find_highest_iou_mask(r['masks'].astype(int),hotplate_index,reference_object_index)
#             # if object mask doesn't inside hotplate, compute by boundingbox
#             if union_score == 0:
#                 object_index = find_highest_iou_bb(r['rois'],hotplate_index,reference_object_index)
            
    
        # select reference object index by score
        reference_object_index, _ = find_highest_score(r['scores'],reference_object_index)
        
        length,max_point,centroid = find_length_of_object(r['masks'].astype(int)[:,:,reference_object_index],r['rois'][reference_object_index][0])
        
        print('object that take to compute ratio:{} with score {}'.format(class_names[r['class_ids'][reference_object_index[0]]],r['scores'][reference_object_index[0]]))
        
        #estimated length of actual objects
        actual_length=0
        if r['class_ids'][reference_object_index[0]] in [1,3]:#fork or spoon
            actual_length=0.2
        elif r['class_ids'][reference_object_index[0]] ==2:#knife
            actual_length=0.2
        
        hotplate_area = np.sum(r['masks'].astype(int)[:,:,hotplate_index])
        
        actual_hotplate_area =(hotplate_area*actual_length**2) / (length**2)
        
        return r,actual_hotplate_area,max_point,centroid
    

In [6]:
def display(image,r,max_point,centroid,class_names,figsize):

    boxes = r['rois']
    masks = r['masks']
    class_ids = r['class_ids']
    scores = r['scores']
    N = boxes.shape[0]
    if not N:
        print("\n*** No instances to display *** \n")
    else:
        assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]
        
        
        
    fig = plt.figure(frameon=False, dpi=100)
    fig.set_size_inches(figsize[0]/1000.0, figsize[1]/1000.0)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.axis('off')

#     _ ,ax =plt.subplots(1,figsize=(16,16))

#     height , width = image.shape[:2]
#     ax.set_xlim(-10,width+10)
#     ax.set_ylim(height+10,-10)

    masked_image = image.astype(np.uint32).copy()


    for i in range(N):
        class_id = class_ids[i]


        #draw boundingbox
        mask = r['masks'][:,:,i]
        padded_mask = np.zeros(
                    (mask.shape[0] + 2, mask.shape[1] + 2), dtype=np.uint8)
        padded_mask[1:-1, 1:-1] = mask[:,:]
        contours = find_contours(padded_mask,0.5)
        contours_len = len(contours[0])
        tolerance = get_tolerance(class_names[class_id],contours_len)
        coords = approximate_polygon(contours[0],tolerance=tolerance)# tolerance need to compute based on size of image
        ax.plot(coords[:, 1], coords[:, 0], '-g', linewidth=0.1)



        # draw text
        score = scores[i] if scores is not None else None
        label = class_names[class_id]
        caption = "{} {:.3f}".format(label, score) if score else label

        ax.text(coords[:, 1][1], coords[:, 0][1] + 8, caption,
                color='r', size=1, backgroundcolor="none")

    ax.imshow(masked_image.astype(np.uint8))
    img_path = os.path.join("/home/dev02/projects/property_expert_demo/upload/",
                                "seg_" + str(random.randrange(1,100000)) + ".jpg")
    fig.savefig(img_path,  dpi=1000)
    plt.show()
    return img_path

## Run Object Detection

In [7]:
! wget -O test.jpg

wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.


In [8]:

    
# #image to test : use from test1-10
# image = skimage.io.imread('/home/ubuntu/Mask_RCNN/data/train/cracked_window_gg_40.jpg')
# # image1 = skimage.io.imread('/home/ubuntu/Mask_RCNN/samples/coco/test.jpg')
# threshold = 0.9

# #seleceted image : 

# # Run detection
# start = time.time()
# results = model.detect([image], verbose=1)

# # Visualize results
# r = results[0]
# # r ,actual_hotplate_area,max_point,centroid = get_ratio_and_refine_hotplate(r,class_names,threshold) 


# visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'],  class_names, r['scores'],max_point = [0,0],centroid = [0,0])
    
# h,w,_ = image.shape    
    

# # display(image,r,max_point,centroid,class_names,[w,h])
# # print('Area:',actual_hotplate_area)
# print('Total in:',time.time()-start)

In [9]:
# y1,x1,y2,x2 = r['rois'][0]
# img = cv2.rectangle(image, (x1, y1), (x2, y2), (255,0,0), 2)
# plt.imshow(img)

In [10]:
# # print(len(annotations[0]['regions']))
# # print(annotations[0]['regions'])
# # print(annotations)
# img_name = 'hotplate_crack_290.jpg'
# all_points_x=[]
# all_points_y=[]
# for a in gt:
#     if a['filename']==img_name : 
#         regions = a['regions']
#         for region in regions.values():
#             all_points_x = list(map(int, region['shape_attributes']['all_points_x']))
#             all_points_y = list(map(int, region['shape_attributes']['all_points_y']))
#             bboxnp.min(all_points_y),np.min(all_points_x),np.max(all_points_y),np.max(all_points_x)
#         break

In [58]:
def compute_iou(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    yA = np.maximum(boxA[0], boxB[0])
    xA = np.maximum(boxA[1], boxB[1])
    yB = np.minimum(boxA[2], boxB[2])
    xB = np.minimum(boxA[3], boxB[3])
    print('ya xa yb cb boxA boxB',yA,xA,yB,xB,boxA,boxB)

    # compute the area of intersection rectangle
    interArea = (xB - xA + 1) * (yB - yA + 1) if (xB - xA + 1) * (yB - yA + 1) > 0 else 0

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = float(interArea) / float(boxAArea + boxBArea - interArea) 
    print('vaicalon',iou)
    # return the intersection over union value
    return iou

In [59]:
import json
# global TP,FP,FN
test_path = '/home/ubuntu/Mask_RCNN/data/train/'
gt_path = '/home/ubuntu/Mask_RCNN/data/annotations/train.json'

def find_bb(img_name,gt):
    '''
    find bb based on polygon input of grouth truth
    
    input:
    img_name : image name
    gt: ground truth object
    
    return :
    boudning box or None
    '''
    all_points_x=[]
    all_points_y=[]
    for a in gt:
        if a['filename']==img_name : 
            regions = a['regions']
            bb_gt = []
            for region in regions.values():
                all_points_x = list(map(int, region['shape_attributes']['all_points_x']))
                all_points_y = list(map(int, region['shape_attributes']['all_points_y']))
                bb = [np.min(all_points_y),np.min(all_points_x),np.max(all_points_y),np.max(all_points_x)]
                bb_gt.append(bb)
            return bb_gt
    return None


def f1_score(test_path,gt_path,threshold):
    '''
    input :
    test_path : path to test images folder
    gt_path: path to ground truth folder
    threshold : threshold for iou
    
    return:
    f1 score
    precision score
    recall score
    '''
    # init
    TP, FP ,FN = 0 ,0 ,0

    test_list = os.listdir(test_path)    
    gt = json.load(open(gt_path))
    gt = [a for a in gt if a['regions']]

    img_count = 0 #count number of img take into compute Precision/Recall
    for idx,img_path in enumerate(test_list):
        # sanity check file's extension
        if '.jpg' in img_path:
            pass
        else:
            continue


        img_count+=1
        img_name = img_path.split('/')[-1]
        if img_name =='Antwort_VP_20140123T200147_Lorbeer_5.Schaden_gesamt2.jpg':
            image = skimage.io.imread(os.path.join(test_path,img_name))


            # init count variable in order to compute FP, FN
            gt_detected,detect_detected = [],[]

            # detect
            r = model.detect([image], verbose=0)[0]
            detect_bbs= r['rois']

            # get groundtruth 
            gt_bbs = find_bb(img_name,gt)
            # if don't have grouth truth so any detection bb is FP
            if gt_bbs is None :
                FP += len(detect_bbs)
                continue
            else:
                pass

            # NOTE :don't have to sort because the result is sorted

            for gt_idx,gt_bb in enumerate(gt_bbs):
                for detect_idx ,detect_bb in enumerate(detect_bbs):

                    iou = compute_iou(gt_bb,detect_bb)

                    # if this bb has claimed as one of gt so skip
                    if (gt_idx in gt_detected) or (detect_idx in detect_detected):
                        continue

                    if iou >= threshold:
#                         print(iou,gt_bb,detect_bb)
                        gt_detected.append(gt_idx)
                        detect_detected.append(detect_idx)
                        TP+=1


            # FP : any detected bb isn't have iou with gt > threshold
            FP += len(detect_bbs) - len(gt_detected)
            # FN : any groundtruth objects that haven't detected
            FN += len(gt_bbs) - len(detect_detected)
            print('TP,FP,FN',TP,FP,FN)
            print(len(detect_bbs))
            break

#         precision = TP/( TP +FP )
#         recall = TP/(TP + FN)
#         f1 = precision*recall/(precision+recall)
#         print('Tested on {} images with f1={} precision={} recall={} '.format(img_count,f1,precision,recall))
#         return f1 , precision , recall





In [60]:
threshold=0.5
f1_score(test_path,gt_path,threshold)

/home/ubuntu/anaconda3/envs/mask/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


ya xa yb cb boxA boxB 877 1270 1004 1226 [805, 1118, 1004, 1226] [ 877 1270 1361 2493]
vaicalon 0.0
ya xa yb cb boxA boxB 805 1177 873 1226 [805, 1118, 1004, 1226] [ 544 1177  873 1622]
vaicalon 0.02084214341811152
ya xa yb cb boxA boxB 1459 2688 1004 1226 [805, 1118, 1004, 1226] [1459 2688 1536 2832]
vaicalon -1.0525402104782096
ya xa yb cb boxA boxB 805 1118 1004 857 [805, 1118, 1004, 1226] [  54   60 1501  857]
vaicalon 0.0
ya xa yb cb boxA boxB 805 2184 837 1226 [805, 1118, 1004, 1226] [   0 2184  837 2905]
vaicalon 0.0
ya xa yb cb boxA boxB 1343 1118 1004 693 [805, 1118, 1004, 1226] [1343  170 1886  693]
vaicalon 0.8762901726752433
ya xa yb cb boxA boxB 1066 1118 1004 461 [805, 1118, 1004, 1226] [1066   24 1513  461]
vaicalon 0.22479888544335086
ya xa yb cb boxA boxB 1750 1118 1004 967 [805, 1118, 1004, 1226] [1750   88 1976  967]
vaicalon 1.017666879154904
ya xa yb cb boxA boxB 805 2029 545 1226 [805, 1118, 1004, 1226] [   0 2029  545 2450]
vaicalon 4.668449678608352
ya xa yb cb 

In [80]:
scores = r['scores']
print(scores)
print(np.argsort(scores))

[0.99972516 0.9994887  0.99626464 0.9947519  0.9917522  0.9287445
 0.9239769  0.9204836  0.875345   0.8752513  0.7477828  0.71151817]
[11 10  9  8  7  6  5  4  3  2  1  0]
